In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
sber_data = pd.read_csv('data/sber_data.csv', sep=',')

## Дубликаты
Дубликатами называются записи, для которых значения (всех или большинства) признаков совпадают. 

## Обнаружение и ликвидация дубликатов
Сравним число уникальных значений в столбце id с числом строк. Число уникальных значений вычислим с помощью метода nunique()

In [10]:
sber_data['id'].nunique() == sber_data.shape[0]

True

Чтобы отследить дубликаты, можно воспользоваться методом duplicated(), который возвращает булеву маску для фильтрации. Для записей, у которых совпадают признаки, переданные методу, он возвращает True, для остальных — False.

У метода есть параметр subset — список признаков, по которым производится поиск дубликатов. По умолчанию используются все столбцы в DataFrame и ищутся полные дубликаты.

Найдём число полных дубликатов таблице sber_data. Предварительно создадим список столбцов dupl_columns, по которым будем искать совпадения (все столбцы, не включая id). 

Создадим маску дубликатов с помощью метода duplicated() и произведём фильтрацию. Результат заносим в переменную sber_duplicates. Выведем число строк в результирующем DataFrame:

In [11]:
dupl_columns = list(sber_data.columns)
dupl_columns.remove('id')

mask = sber_data.duplicated(subset=dupl_columns)
sber_duplicates = sber_data[mask]
print(f'Число дубликатов: {sber_duplicates.shape[0]}')

Число дубликатов: 562


Теперь нам необходимо от них избавиться. Для этого легче всего воспользоваться методом drop_duplicates(), который удаляет повторяющиеся записи из таблицы. 

✍️ Создадим новую таблицу sber_dedupped, которая будет версией исходной таблицы, очищенной от полных дубликатов.

In [12]:
sber_dedupped = sber_data.drop_duplicates(subset=dupl_columns)
print(f'Результатирующее число записей: {sber_dedupped.shape[0]}')

Результатирующее число записей: 29909


## Неинформативные признаки

Неинформативными называются признаки, в которых большая часть строк содержит одинаковые значения (например, пол клиентов в мужском барбершопе), либо наоборот — признак, в котором для большинства записей значения уникальны (например, номер телефона клиента). 

## Обнаружение и их ликвидация
Неинформативным считается признак, у которого 95% и выше одинаковых значений

Найдем неинформативные признаки:
 - Создаем пустой список low_information_cols для неинформативных признаков
 - В цикле пройдемся по всем именам столбцов таблицы и для каждого будем совершать действия:
    - Рассчитаем top_freq - наибольшую относительную частоту методом value_counts(normalize=True) - вернется доля уникальных значений, нам нужен его максимум
    - Рассчитаем nunique_ratio - отношение числа уникальных значений в столбце к размеру всего столбца. Число уникальных значений получим методом nunique(), a размер признака - count()
    - Сравним каждое из полученных чисел с пороговым значением и добавим в список неинформативных признаков

In [14]:
# Список неинформативных признаков
low_information_cols = []

# Цикл по всем столбцам
for col in sber_data.columns:
    # Наибольшая относительная частота в признаке
    top_freq = sber_data[col].value_counts(normalize=True).max()
    # Доля уникальных значений от размера признака
    nunique_ratio = sber_data[col].nunique() / sber_data[col].count()
    # Сравниваем наибольшую частоту с порогом
    if top_freq > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    # Сравниваем долю уникальных значений с порогом
    if nunique_ratio > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')

id: 100.0% уникальных значений
oil_chemistry_raion: 99.03% одинаковых значений
railroad_terminal_raion: 96.27% одинаковых значений
nuclear_reactor_raion: 97.17% одинаковых значений
big_road1_1line: 97.44% одинаковых значений
mosque_count_1000: 98.08% одинаковых значений


Итак, мы нашли шесть неинформативных признаков. Теперь можно удалить их с помощью метода drop(), передав результирующий список в его аргументы.

In [15]:
information_sber_data = sber_data.drop(low_information_cols, axis=1)
print(f'Результатирующее число признаков: {information_sber_data.shape[1]}')

Результатирующее число признаков: 55


## Важность признаков
Это влияние на целевой признак

Признаки, обладающие низкой важность называются нерелевантными